In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [6]:
value = """
N_CLASSES = 102
SHUFFLE_BUFFER_SIZE = 500

# Augmentation Configuration
CROP_LOW_LIMIT = int(.08*IMG_SIZE)
CROP_HIGH_LIMIT = IMG_SIZE+1
PIXEL_MAX_VAL = 255
RAND_GENERATOR = tf.random.Generator.from_seed(SEED)

def random_choice(fn1, fn2, p=0.5):
    choice = RAND_GENERATOR.uniform(())
    return fn1() if choice<=p else fn2()


def random_apply(fn, arg, p=.5):
    choice = RAND_GENERATOR.uniform(())
    return fn(arg) if choice<=p else arg

def random_crop_and_resize(image):
    def crop():
        dim1 = tf.random.uniform(
            [],
            minval=CROP_LOW_LIMIT,
            maxval=CROP_HIGH_LIMIT,
            dtype=tf.int32
        )
        dim2 = int((3/4)*float(dim1))

        shape = random_choice(
            lambda: (dim1, dim2, image.shape[-1]),
            lambda: (dim2, dim1, image.shape[-1]),
        )
        cropped = tf.image.random_crop(image, shape)
        return cropped
    
    cropped = crop()
    resized = tf.image.resize(cropped, (IMG_SIZE, IMG_SIZE))
    return resized

def color_distortion(image, factor=0.8):
    def color_jitter(x):
        x = tf.image.random_brightness(x, int(factor*PIXEL_MAX_VAL))
        x = tf.image.random_contrast(x, 1-factor, 1+factor)
        x = tf.image.random_saturation(x, 1-factor, 1+factor)
        x = tf.image.random_hue(x, 0.2)
        x = tf.clip_by_value(x, 0, PIXEL_MAX_VAL+1)
        return x

    def color_drop(x):
        x = tf.image.rgb_to_grayscale(x)
        x = tf.tile(x, [1, 1, 3])
        return x

    # randomly apply transformation with probability p.
    image = random_apply(color_jitter, image, p=0.8)
    image = random_apply(color_drop, image, p=0.2)
    return image

def gaussian_blur(image, kernel_size=IMG_SIZE//10):
    blurred = tfa.image.gaussian_filter2d(
        image,
        (kernel_size, kernel_size),
        (0.1, 2.0)
    )
    return blurred

@tf.function
def apply_augmentations_for_self_supervision(image):
    def random_flip(x):
        return tf.image.random_flip_left_right(x)

    image = random_crop_and_resize(image)
    image = random_flip(image)
    image = color_distortion(image)
    image = random_apply(gaussian_blur, image, 0.5)
    return image

def make_display_input(image):
    transform1 = apply_augmentations_for_self_supervision(image)
    transform2 = apply_augmentations_for_self_supervision(image)
    return image, transform1, transform2

def make_contrastive_input(image, label):
    transform1 = apply_augmentations_for_self_supervision(image)
    transform2 = apply_augmentations_for_self_supervision(image)
    inputs = tf.stack([transform1, transform2], axis=0)
    labels = tf.stack([label, label], axis=0)
    return inputs, labels
"""

key = 'color_distortion'

print(shelf.get(key))
shelf[key] = value
shelf.sync()

None


## Enumerate

In [7]:
keys = list(shelf.keys())
print(len(keys), keys)

17 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples', 'draw_bb_boxes', 'dense_boxes', 'save_frames_as_gif', 'display_gifs', 'display_gifs_ipython', 'openai_virtual_display', 'display_images_in_grid', 'plot_loss', 'color_distortion']


## Query

In [8]:
print(shelf['color_distortion'])


N_CLASSES = 102
SHUFFLE_BUFFER_SIZE = 500

# Augmentation Configuration
CROP_LOW_LIMIT = int(.08*IMG_SIZE)
CROP_HIGH_LIMIT = IMG_SIZE+1
PIXEL_MAX_VAL = 255
RAND_GENERATOR = tf.random.Generator.from_seed(SEED)

def random_choice(fn1, fn2, p=0.5):
    choice = RAND_GENERATOR.uniform(())
    return fn1() if choice<=p else fn2()


def random_apply(fn, arg, p=.5):
    choice = RAND_GENERATOR.uniform(())
    return fn(arg) if choice<=p else arg

def random_crop_and_resize(image):
    def crop():
        dim1 = tf.random.uniform(
            [],
            minval=CROP_LOW_LIMIT,
            maxval=CROP_HIGH_LIMIT,
            dtype=tf.int32
        )
        dim2 = int((3/4)*float(dim1))

        shape = random_choice(
            lambda: (dim1, dim2, image.shape[-1]),
            lambda: (dim2, dim1, image.shape[-1]),
        )
        cropped = tf.image.random_crop(image, shape)
        return cropped
    
    cropped = crop()
    resized = tf.image.resize(cropped, (IMG_SIZE, IMG_SIZE))
 